# t-SNE

The goal of this assignment is to train a Word2Vec skip-gram model over [Text8](http://mattmahoney.net/dc/textdata) data

#### libraries load up

In [2]:
%matplotlib inline

In [3]:
from __future__ import print_function #https://www.python.org/dev/peps/pep-3105/
#Make print a function

In [4]:
import collections #https://docs.python.org/3/library/collections.html
#container datatypes, alternatives to built-in containers, dict, list, set, and tuple.
import os #https://docs.python.org/3/library/os.html
#a portable way of using operating system dependent functionality
from os import path #https://docs.python.org/3/library/os.path.html#module-os.path
#functions on pathnames
import zipfile #https://docs.python.org/3/library/zipfile.html
#Work with ZIP archives
import urllib.request 
#https://docs.python.org/3.0/library/urllib.request.html#module-urllib.request

In [5]:
import tensorflow as tf
import numpy as np

In [6]:
import math
import random #https://docs.python.org/3/library/random.html
#pseudo-random number generators

In [7]:
from matplotlib import pylab

In [8]:
from sklearn.manifold import TSNE

In [28]:
#from six.moves import range
#from six.moves.urllib.request import urlretrieve

#### data load up

In [9]:
path = 'C:/Users/Gavrilov/My Projects/t-SNE/'
url = 'http://mattmahoney.net/dc/'
file_name = 'text8.zip'
expected_bytes = 31344016

In [10]:
full_file_path = os.path.join(path, file_name)
#https://docs.python.org/3/library/os.path.html#os.path.join

In [11]:
def get_file(path, file_name, expected_bytes):
    if not os.path.exists(full_file_path):
    #https://docs.python.org/3/library/os.path.html#os.path.exists
    #Return True if path refers to an existing path or an open file descriptor
        file, headers = urllib.request.urlretrieve(url+file_name, full_file_name)
        #https://docs.python.org/3.0/library/urllib.request.html#urllib.request.urlretrieve
    if os.path.exists(full_file_path):
        file = full_file_path
    stat_info = os.stat(full_file_path)
    #https://docs.python.org/3/library/os.html#os.stat
    if stat_info.st_size == expected_bytes:
        print('File found, size verified %s' % full_file_path) #advance printing
    else:
        raise Exception ('Unable to verify a file ' + file_name + ' with a size of ' + str(stat_info.st_size))
    return file

In [12]:
file = get_file(path, file_name, expected_bytes)

File found, size verified C:/Users/Gavrilov/My Projects/t-SNE/text8.zip


#### data extract

In [13]:
with zipfile.ZipFile(file) as f:
    #words = tf.compat.as_str(f.read(f.namelist()[0])).split()
    words = f.namelist()[0] #https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile.namelist
    #Return a list of archive members by name
    #Extracting the first file enclosed in a zip file
    words = f.read(words) #https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile.read
    #Return the bytes of the file name in the archive
    words = tf.compat.as_str(words).split() #https://www.tensorflow.org/api_docs/python/tf/compat
    #converts an object to a str
    #split to a list of words

In [14]:
len(words)

17005207

#### data manipulation

In [102]:
vocab_size = 500000

###### creating list of most common words

In [103]:
count = [['UNK', -1]] #list of pairs of form ['word', number_of_occurences]
#-1 is apparently a placeholder value
count.extend(collections.Counter(words).most_common(vocab_size-1))
#https://docs.python.org/3/library/collections.html#collections.Counter
#https://docs.python.org/3/library/collections.html#collections.Counter.most_common
#https://docs.python.org/3.6/tutorial/datastructures.html
#appending all the items from the iterable

In [104]:
count[0:2]

[['UNK', -1], ('the', 1061396)]

###### building the dictionary 

In [105]:
#converting most common words into dictionary 
words_most_common_dict = {word:idx for word, idx in count}

##### reverse the dictionary

In [106]:
words_most_common_dict_reverse = {idx:word  for word, idx in words_most_common_dict.items()}

##### creating list of indices

In [107]:
unk_count = 0

In [108]:
data = []

In [109]:
for word in words:
    if word in words_most_common_dict:
        index = words_most_common_dict[word]
    else:
        index = 0  # dictionary['UNK']
        unk_count = unk_count + 1
    data.append(index)

In [110]:
data[0:2]

[303, 572]

##### replacing of rare words with UNK token unknown word

In [111]:
count[0][1] = unk_count
#-1 placeholder value from [['UNK', -1]] filled with count[0][1] = unk_count

In [112]:
count[0][1]

0

#### generating a training batch for the skip-gram model

In [113]:
data_index = 0

In [114]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    #https://docs.python.org/3/reference/simple_stmts.html#the-assert-statement
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    #https://docs.python.org/3/library/collections.html#collections.deque
    for _ in range(span):
    #_ so-called “I don’t care”
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [115]:
for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)

In [116]:
batch, labels

(array([131815, 131815, 131815, 131815, 325873, 325873, 325873, 325873]),
 array([[   572],
        [  7219],
        [   303],
        [325873],
        [593677],
        [131815],
        [  7219],
        [   572]]))

#### train a skip-gram model

In [117]:
batch_size = 128
embedding_size = 128 #Dimension of the embedding vector.
skip_window = 1 #How many words to consider left and right.
num_skips = 2 #How many times to reuse an input to generate a label.
#We pick a random validation set to sample nearest neighbors. here we limit the
#validation samples to the words that have a low numeric ID, which by
#construction are also the most frequent. 
valid_size = 16 #Random set of words to evaluate similarity on.
valid_window = 100 #Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 #Number of negative examples to sample.

In [98]:
graph = tf.Graph()
with graph.as_default(), tf.device('/cpu:0'):
    #Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    #https://www.tensorflow.org/api_docs/python/tf/placeholder
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    #https://www.tensorflow.org/versions/master/api_docs/python/tf/constant
  
    #Variables.
    embeddings = tf.Variable(
        tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(
        tf.truncated_normal([vocab_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocab_size]))
  
    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
            labels=train_labels, num_sampled=num_sampled, num_classes=vocab_size))

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [118]:
num_steps = 100001

In [119]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
    if step % 2000 == 0:
        if step > 0:
            average_loss = average_loss / 2000
        # The average loss is an estimate of the loss over the last 2000 batches.
        print('Average loss at step %d: %f' % (step, average_loss))
        average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
        sim = similarity.eval()
        for i in range(valid_size):
            valid_word = words_most_common_dict_reverse[valid_examples[i]]
            top_k = 8 # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k+1]
            log = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = words_most_common_dict_reverse[nearest[k]]
                log = '%s %s,' % (log, close_word)
            print(log)
    final_embeddings = normalized_embeddings.eval()

Initialized


InvalidArgumentError: indices[14] = 1061396 is not in [0, 50000)
	 [[Node: embedding_lookup = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@Variable"], validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable/read, _arg_Placeholder_0_0)]]

Caused by op 'embedding_lookup', defined at:
  File "C:\Users\Gavrilov\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Gavrilov\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Gavrilov\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Gavrilov\Anaconda3\lib\site-packages\zmq\eventl